In [54]:
import json
import os
import platform
from datetime import datetime
import time
from datetime import datetime

import pandas as pd
from tqdm import tqdm
import yaml
from IPython.display import display, Markdown
from git import Repo as RepoLoader  # gitpython
from selenium.webdriver import Chrome

import papers_analyser.db as db
from papers_analyser import scrape_papers_with_code as navigator
from papers_analyser.paper import Paper, get_repo, scrape_paper, load_papers, __load_json_files, __dict_to_paper

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
project_dir = os.path.abspath("..")

## Connect to DB

In [86]:
connection = db.create_connection( project_dir + "/data/paper.db")
db.create_table_if_not_exist(connection)

2.6.0


## Github API auth-token

In [66]:
auth_token = yaml.load(open("config.yaml", "r"))["auth_token"]

e:\programme\reproduce-ml-papers\venv\lib\site-packages\ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


# Scrape papers
Scraping papers on Github and Paperswithcode.com. Only query those, that have not been inserted into the db.

Seperation of `repos` and `papers` because weird behaviour of paperswithcode.com: Sometimes mixing on-site links with github links, which makes absolutely no sense.
Get auth-token from your github profile under personal tokens.

In [85]:
json_path = project_dir + "/data"
since_date = datetime.strptime("2020-01-01", '%Y-%m-%d')


In [87]:
papers= load_papers(project_dir + "/data",  datetime.strptime("2020-01-01", '%Y-%m-%d'))


In [89]:
for paper in tqdm(papers):
    for repo in paper.repos:
        if not repo.scraped:
            repo.scrape(auth_token)

## Take a look on README files

In [22]:
readmes = [paper.repo[0].readme for paper in papers]

NameError: name 'papers' is not defined

In [ ]:
display(Markdown(readmes[4]))

# Save to database

In [90]:
for paper in papers:
    if paper.repos_scraped():
        db.insert_paper(connection,paper)

connection.commit()

In [28]:
connection.close()

## Clone repo

In [36]:
dir = driverPath = project_dir + "/repos"
for paper in papers:
    paper_path = dir + "/" + paper.title.replace( ":","")
    if not os.path.exists(paper_path):
        os.mkdir(paper_path)
        
    for repo in paper.repo:
        repo_path = paper_path+ repo.repo_name.replace("/","_")
        
        if not os.path.exists(repo_path):
            os.mkdir(repo_path)
            
        RepoLoader.clone_from(repo.clone_url,repo_path)
    

